In [ ]:
!python --version

In [ ]:
from marcap import marcap_data
from kafka import KafkaProducer
from json import dumps
import time
import pandas as pd

# 카프카 프로듀서 객체 생성
producer = KafkaProducer(acks=0, 
                         bootstrap_servers=['ka01:9092', 'ka02:9092', 'ka03:9092'],
                         value_serializer=lambda x: dumps(x, ensure_ascii = False).encode('utf-8'))
dt_index=pd.date_range(start='1995-05-02', end='2020-12-31')
dt_list=dt_index.strftime("%Y-%m-%d").tolist()
start = time.time()

# 일별 국내 시가총액 TOP 20 json 데이터 생성
for i in dt_list:
    df = marcap_data(i)
    
    # 주식 장이 열리지 않은 날은 빈 값이므로 생략
    if len(df.values) == 0:
        print(i+"데이터 없음")
    
    else:
        # 20위까지
        df2=df[(df['Rank'] <= 20)]
        # all Nan이어서 삭제
        df3=df2.drop(['Dept'], axis=1)
        data=df3.to_dict('records')

        for j in data:
            j['Date']=i
            producer.send('Stock_Data', value=j)
            producer.flush()
            
# 초당 몇 건 처리하였는지 확인
print("elapsed :", time.time() - start)